In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
import pywt
import numpy as np
import imageio
import tensorflow as tf
import cv2
import time
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
from tensorflow.keras.models import Model, load_model
from sklearn.metrics import classification_report
import os

In [2]:
def split_eletrodes(data):
    lb = data[0]
    sign_eletrode = []
    index_1 = 1
    index_2 = 51

    for e in eletrodes:
        sign_eletrode.append(data[index_1:index_2])
        index_1 += 50
        index_2 += 50
    
    return lb, sign_eletrode

def plot_graph(lb, features, eletrode):
    title = labels[int(lb)]
    index_eletrode = eletrodes.index(eletrode)

    plt.clf()
    plt.title(eletrode + ' - ' + title)
    plt.plot(features[index_eletrode], linestyle="-", marker="x")
    plt.show()



In [3]:
labels = [
    'left',
    'right'
]
eletrodes = ['F3', 'F1', 'Fz', 'F2', 'F4', 'FC5', 'FC3', 'FC1', 'FCz',
             'FC2', 'FC4', 'FC6', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4',
             'C6', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6',
             'O1', 'O2']


In [4]:
data = pd.read_csv('dataset/sp1s_aa_train.csv', delimiter=";")


In [9]:
for eletrode in eletrodes:
    widths = np.arange(1, 51)
    indx = eletrodes.index(eletrode)
    cont = 0 
    cwt_features = []
    label_list = []
    for row in data.iloc:
        lb, sign = split_eletrodes(row.to_numpy())
        cwtmatr = (signal.cwt(sign[indx], signal.morlet, widths))
        img = np.zeros((np.array(cwtmatr).shape[0], np.array(cwtmatr).shape[1], 3))
        
        img[:,:,0] = abs(cwtmatr)
        img[:,:,1] = abs(cwtmatr)
        img[:,:,2] = abs(cwtmatr)
        

        label_list.append(int(lb))
        cwt_features.append(img)

    features_train = []
    labels_train = []
    features_test = []
    labels_test = []

    for i in range(len(cwt_features)):
        if(i < 80):
            features_test.append(cwt_features[i])
            labels_test.append(label_list[i])
        else:
            features_train.append(cwt_features[i])
            labels_train.append(label_list[i])

    features_train = np.array(features_train).reshape((236, 50, 50, 3))
    features_test = np.array(features_test).reshape((80, 50, 50, 3))
    img_generator = tf.keras.preprocessing.image.ImageDataGenerator()
    features_train = img_generator.flow(
        features_train,
        y=np.array(labels_train),
        batch_size=16,
        
        shuffle=True)
    features_test = img_generator.flow(
        features_test,
        y=np.array(labels_test),
        batch_size=16,
        
        shuffle=True)
    conv_base = tf.keras.applications.EfficientNetB3(
                  include_top=False,
                  input_shape=(50, 50, 3),
                  pooling='avg'
                  )
    input_layer=tf.keras.Input(shape=(50,50,3))

    x=conv_base(input_layer,training=True)

    output=tf.keras.layers.Dense(units=1,activation='sigmoid')(x)
    model=tf.keras.Model(input_layer,output)
    model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(),metrics=['accuracy'])
    history = model.fit(features_train.x, features_train.y, epochs=10, 
                    validation_data=(features_test.x, features_test.y))
  
    cwt_features_val = np.array(cwt_features).reshape((316, 50, 50, 3))
    labels_val = np.array(label_list)
    
    cont_predicoes = 0
    cont_acertos = 0
    label_0 = 0
    label_1 = 0
    acertos_0 = 0
    acertos_1 = 0

    for i in range(len(cwt_features_val)):
        x = cwt_features_val[i].reshape(1,50,50,3)
        y = labels_val[i]
        y_predict = model.predict(x)
        y_predict = [1 * (x[0]>=0.5) for x in y_predict]
        
        if(y == 0):
            label_0 += 1
            print(y_predict)
            if(y_predict == [0]):
                acertos_0 += 1
                cont_acertos +=1
        else:
            label_1 += 1
            print(y_predict)
            if(y_predict == [1]):
                acertos_1 += 1
                cont_acertos +=1
        cont_predicoes+=1

    f = open("metricas_por_eletrodos3.txt", "a")
    txt_metricas =eletrode+';'+str(cont_acertos)+';'+str(cont_predicoes)+';'+str(label_0)+';'+str(acertos_0)+';'+str(label_1)+';'+str(acertos_1)+';'+str(cont_acertos/cont_predicoes)+'\n'
    f.write(txt_metricas)
    f.close()



Epoch 1/10
8/8 [==============================] - 40s 2s/step - loss: 0.7949 - accuracy: 0.5466 - val_loss: 0.9350 - val_accuracy: 0.5375
Epoch 2/10
8/8 [==============================] - 13s 2s/step - loss: 0.7380 - accuracy: 0.6229 - val_loss: 1.0384 - val_accuracy: 0.5375
Epoch 3/10
8/8 [==============================] - 13s 2s/step - loss: 0.5565 - accuracy: 0.7076 - val_loss: 1.1686 - val_accuracy: 0.5250
Epoch 4/10
8/8 [==============================] - 13s 2s/step - loss: 0.5889 - accuracy: 0.8008 - val_loss: 1.3138 - val_accuracy: 0.4875
Epoch 5/10
8/8 [==============================] - 13s 2s/step - loss: 0.4959 - accuracy: 0.7839 - val_loss: 1.2299 - val_accuracy: 0.4625
Epoch 6/10
8/8 [==============================] - 13s 2s/step - loss: 0.3605 - accuracy: 0.8602 - val_loss: 1.2803 - val_accuracy: 0.5125
Epoch 7/10
8/8 [==============================] - 13s 2s/step - loss: 0.4000 - accuracy: 0.8517 - val_loss: 1.5950 - val_accuracy: 0.5375
Epoch 8/10
8/8 [==================

In [75]:

cwt_features_val = np.array(cwt_features).reshape((316, 50, 50, 3))
labels_val = np.array(label_list)
#model = load_model('./models/model.h5')

cont_predicoes = 0
cont_acertos = 0

for i in range(len(cwt_features_val)):
    x = cwt_features_val[i].reshape(1,50,50,3)
    y = labels_val[i]

    y_predict = model.predict(x)
    print(str(y),str(y_predict))
    y_predict = [1 * (x[0]>=0.5) for x in y_predict]
        
    if(y == y_predict):
        cont_acertos +=1
    cont_predicoes+=1
    print(str(y),str(y_predict))
print(cont_acertos, cont_predicoes)



1/1 [==============================] - 0s 93ms/step
0 [[0.57285535]]
0 [1]
1/1 [==============================] - 0s 94ms/step
1 [[0.56339633]]
1 [1]
1/1 [==============================] - 0s 113ms/step
0 [[0.6667988]]
0 [1]
1/1 [==============================] - 0s 97ms/step
1 [[0.6279486]]
1 [1]
1/1 [==============================] - 0s 94ms/step
0 [[0.58380324]]
0 [1]
1/1 [==============================] - 0s 92ms/step
1 [[0.62171066]]
1 [1]
1/1 [==============================] - 0s 94ms/step
0 [[0.5911782]]
0 [1]
1/1 [==============================] - 0s 93ms/step
1 [[0.5152482]]
1 [1]
1/1 [==============================] - 0s 89ms/step
0 [[0.6191803]]
0 [1]
1/1 [==============================] - 0s 96ms/step
0 [[0.64802974]]
0 [1]
1/1 [==============================] - 0s 90ms/step
1 [[0.50028676]]
1 [1]
1/1 [==============================] - 0s 87ms/step
1 [[0.6243213]]
1 [1]
1/1 [==============================] - 0s 96ms/step
0 [[0.65335226]]
0 [1]
1/1 [=========================